<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%B0%B1%ED%85%8C%EC%8A%A4%ED%8A%B8_%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_RAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 첫 설치 이후 설치 안함
!pip install yfinance

In [3]:
# 첫 설치 이후 설치 안함
!pip install pandas_market_calendars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 13.0 MB/s eta 0:00:00


In [4]:
# 첫 설치 이후 설치 안함
!pip install fredapi

In [52]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv
from fredapi import Fred

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
today = datetime.today().date()
# start_date = today - timedelta(days=375)

# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 전략명
str_strategy = 'RAA '

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} backtest {str_strategy}.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')

# 월평균영업일수. average Monthly Business Day
MBD = 21

# 전략에 사용할 ETF 명 리스트에 담기
etf_stock = ['QQQ', 'IWN']
etf_bond = ['IEF', 'TLT']
etf_real = ['GLD']
etf_canary = ['VWO', 'BND']

# 자산군 리스트 더하기, 중복 제거
etf_order = list(set(etf_stock + etf_bond + etf_real + etf_canary))

# ETF 가격 추출
df_etf = yf.download(etf_order, end=today)['Adj Close']\
.sort_index(ascending=False)[etf_order].rename_axis(None, axis=0).dropna()
display(df_etf)

# 1개월씩 자르기
df_etf_monthly = df_etf[::MBD]
display(df_etf_monthly)

[*********************100%%**********************]  7 of 7 completed


,IWN,IEF,GLD,QQQ,TLT,VWO,BND
2023-09-15,141.0000,93.0900,178.3400,370.8100,92.9600,40.5400,70.7900
2023-09-14,142.3800,93.3800,177.1700,377.2700,93.5300,40.7500,70.9300
2023-09-13,139.9800,93.5900,177.0800,374.2100,94.2100,40.5100,71.0500
2023-09-12,140.9500,93.4800,177.4900,372.7900,94.2300,40.5500,70.9400
2023-09-11,140.7500,93.3500,178.4100,376.9700,93.6900,40.6600,70.8900
...,...,...,...,...,...,...,...
2007-04-16,60.5503,56.2565,68.4000,39.2966,54.2871,27.2092,45.5329
2007-04-13,59.8035,56.1951,67.8400,38.9390,53.9896,26.8727,45.4904
2007-04-12,59.4265,56.2906,66.9900,38.8605,54.1570,26.7853,45.5633
2007-04-11,59.0494,56.2360,67.0800,38.5553,54.1446,26.4488,45.5694


,IWN,IEF,GLD,QQQ,TLT,VWO,BND
2023-09-15,141.0000,93.0900,178.3400,370.8100,92.9600,40.5400,70.7900
2023-08-16,142.6300,93.2015,175.5700,362.5400,93.5594,39.9400,70.5966
2023-07-18,147.8400,96.3088,183.6400,385.7400,101.2166,41.6800,72.3480
2023-06-15,141.6200,96.5639,181.9100,369.7528,102.1332,41.8622,72.3137
2023-05-16,129.2761,97.9593,184.8700,326.7119,101.7437,39.8034,72.5342
...,...,...,...,...,...,...,...
2007-08-09,54.2340,56.9736,65.4600,41.6401,54.2941,29.5875,45.8224
2007-07-11,60.1880,55.6398,65.4400,42.6002,52.8542,31.0533,45.2564
2007-06-11,60.4053,55.1362,64.7000,40.8315,52.1774,28.4097,45.0229
2007-05-10,59.8180,56.8531,66.0000,40.2820,55.0458,27.0539,45.8530


In [26]:
## DataFrame 만들기: 미국 실업률 (08-01: 8월1일~31일 실업률 통계)
fred = Fred(api_key='5958f7e2cfc6adda8b4efaa3c131d32e')
fred_unrate = fred.get_series('UNRATE').sort_index(ascending=False)
df_fred_unrate = pd.DataFrame(fred_unrate, columns=['unemployment rate'])

display(df_fred_unrate)

,unemployment rate
2023-08-01,3.8000
2023-07-01,3.5000
2023-06-01,3.6000
2023-05-01,3.7000
2023-04-01,3.4000
...,...
1948-05-01,3.5000
1948-04-01,3.9000
1948-03-01,4.0000
1948-02-01,3.8000


In [24]:
'''
# 인덱스 만들기
df_momentum_index = ['today/' + str(df_etf.index[0].date()), '1mo_ago/' + str(df_etf.index[MBD].date()), \
                     '3mo_ago/' + str(df_etf.index[MBD*3].date()), '6mo_ago/' + str(df_etf.index[MBD*6].date()), \
                     '12mo_ago/' + str(df_etf.index[MBD*12].date())]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
price_today = df_etf.iloc[0:1]
price_1mo = df_etf.shift(-MBD).iloc[0:1]
price_3mo = df_etf.shift(-MBD*3).iloc[0:1]
price_6mo = df_etf.shift(-MBD*6).iloc[0:1]
price_12mo = df_etf.shift(-MBD*12).iloc[0:1]

df_price = pd.concat([price_today, price_1mo, price_3mo, price_6mo, price_12mo])
df_price.index = df_momentum_index

str_etf_price = str_today + str_strategy + "### ETF 현재-1-3-6-12 개월 가격"
display(Markdown(str_etf_price))
display(df_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'w') as f:
  f.write(f'{str_etf_price}\n')
  df_price.to_csv(f, index=True)
'''

'\n# 인덱스 만들기\ndf_momentum_index = [\'today/\' + str(df_etf.index[0].date()), \'1mo_ago/\' + str(df_etf.index[MBD].date()),                      \'3mo_ago/\' + str(df_etf.index[MBD*3].date()), \'6mo_ago/\' + str(df_etf.index[MBD*6].date()),                      \'12mo_ago/\' + str(df_etf.index[MBD*12].date())]\n\n\n## DataFrame 만들기: 1-3-6-12 개월 etf 가격\nprice_today = df_etf.iloc[0:1]\nprice_1mo = df_etf.shift(-MBD).iloc[0:1]\nprice_3mo = df_etf.shift(-MBD*3).iloc[0:1]\nprice_6mo = df_etf.shift(-MBD*6).iloc[0:1]\nprice_12mo = df_etf.shift(-MBD*12).iloc[0:1]\n\ndf_price = pd.concat([price_today, price_1mo, price_3mo, price_6mo, price_12mo])\ndf_price.index = df_momentum_index\n\nstr_etf_price = str_today + str_strategy + "### ETF 현재-1-3-6-12 개월 가격"\ndisplay(Markdown(str_etf_price))\ndisplay(df_price)\n\n# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함\nwith open(output_path + filename, \'w\') as f:\n  f.write(f\'{str_etf_price}\n\')\n  df_price.to_csv(f, index=True)\n'

In [94]:
row_count = len(df_etf_monthly.index)

df_etf_monthly_copy = df_etf_monthly.copy(deep=True)
df_etf_momentum_score = pd.DataFrame(index=df_etf_monthly.index, columns=df_etf_monthly.columns)

## DataFrame 만들기: 모멘텀 스코어
for i in range(0, row_count-12):
  ror_1mo = df_etf_monthly_copy.pct_change(-1).iloc[0]*100   # 1개월 수익률 * 가중치
  ror_3mo = df_etf_monthly_copy.pct_change(-3).iloc[0]*100    # 3개월 수익률 * 가중치
  ror_6mo = df_etf_monthly_copy.pct_change(-6).iloc[0]*100   # 6개월 수익률 * 가중치
  ror_12mo = df_etf_monthly_copy.pct_change(-12).iloc[0]*100  # 12개월 수익률 * 가중치
  df_etf_monthly_copy.drop(df_etf_monthly_copy.index[0], inplace=True)

  # 가중치 더하기
  df_etf_momentum_score.iloc[i] = (12 * ror_1mo) + (4 * ror_3mo) + (2 * ror_6mo) * (ror_12mo)

df_etf_momentum_score.dropna(inplace=True)

str_etf_momentum_score = str_today + str_strategy + "### ETF 별 1-3-6-12 개월 Momentum Score"
display(Markdown(str_etf_momentum_score))
display(df_etf_momentum_score)
'''
# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_momentum_score}\n')
  df_etf_momentum_score.to_csv(f, index=True)
'''

### 2023-09-16 RAA ### ETF 별 1-3-6-12 개월 Momentum Score

,IWN,IEF,GLD,QQQ,TLT,VWO,BND
2023-09-15,-11.4121,9.5712,7.7532,1138.1253,187.4194,40.2486,-2.3756
2023-08-16,87.7155,-25.5741,-52.4485,328.5353,148.8924,-35.9680,-27.4798
2023-07-18,99.3089,-0.2559,91.9081,2631.8334,46.1071,-2.5883,1.0696
2023-06-15,129.6800,-17.8320,114.1274,2269.1639,14.2366,146.6846,-0.5455
2023-05-16,75.5379,10.4168,259.2974,316.8997,-127.2292,-35.8016,1.8866
...,...,...,...,...,...,...,...
2008-08-08,99.0390,13.4627,-548.8867,37.9926,-3.0395,-50.9735,-16.1682
2008-07-10,117.7006,105.1105,811.9570,-7.1879,75.2744,17.8615,30.8072
2008-06-10,192.2374,29.0941,500.7209,5.3370,-18.8935,-353.0374,-13.8941
2008-05-09,172.4482,143.6445,336.2213,23.9844,54.6408,-185.2118,66.3211


"\n# csv에 제목 및 DataFrame 추가\nwith open(output_path + filename, 'a') as f:\n  f.write('\n')\n  f.write(f'{str_etf_momentum_score}\n')\n  df_etf_momentum_score.to_csv(f, index=True)\n"

In [22]:
## DataFrame 만들기: 모멘텀 스코어
df_etf_momentum_score = (12 * ror_1mo) + (4 * ror_3mo) + (2 * ror_6mo) + (ror_12mo)
df_etf_momentum_score.index = ['momentum_score']

str_etf_momentum_score = str_today + str_strategy + "### ETF 별 1-3-6-12 개월 Momentum Score"
display(Markdown(str_etf_momentum_score))
display(df_etf_momentum_score)
'''
# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_momentum_score}\n')
  df_etf_momentum_score.to_csv(f, index=True)
'''

### 2023-09-16 RAA ### ETF 별 1-3-6-12 개월 Momentum Score

,IWN,IEF,GLD,QQQ,TLT,VWO,BND
momentum_score,-6.7326,-26.7074,23.7483,97.0971,-75.3885,18.5638,-9.4723


In [ ]:
## DataFrame

# 지금 실업률이 1년전 실업률보다 높은가? (낮으면 5개 자산 투자. 주식 40%, 채권 40%, 실물자산 20%)
df_fred_unrate
# 높으면 VWO, BND 1-3-6-12 모멘텀스코어가 하나라도 마이너스인가?

# 하나라도 마이너스이면 채권 포트폴리오 투자 (IEF 50%, TLT 50%)

# 둘 다 플러스이면 5개 자산 비율 투자


In [ ]:
# 최근 6개월 상위 수익 5개
monthly_index = df_etf_ror.index
high_name = ['1st', '2nd', '3rd', '4th', '5th']

high_etf = df_etf_ror.apply(lambda row: row.nlargest(5).index.tolist(), axis=1)
df_high_etf = pd.DataFrame(high_etf.to_list(), columns=high_name, index=monthly_index)
display(df_high_etf)

etf_ror_list = []
row_count = len(high_etf)

for i in range(row_count):
  a_list = df_etf_ror.iloc[i][df_high_etf.iloc[i]].tolist()
  etf_ror_list.append(a_list)

df_high_ror = pd.DataFrame(etf_ror_list, columns=high_name, index=monthly_index)
display(df_high_ror)

'''
# 개수만큼 선정
row_count = df_etf_ror.shape[0]
monthly_index = df_etf_ror.index

df_high_etf = pd.DataFrame(columns=[i for i in range(row_count)], index=monthly_index)

row_count = len(df_high_etf)

for i in range(row_count):
  df_high_etf.iloc[i] = df_etf_ror.columns

display(df_high_etf)
'''

,1st,2nd,3rd,4th,5th
2023-09-13,SPY,EWJ,DBC,EEM,VGK
2023-08-14,SPY,EWJ,VGK,GLD,DBC
2023-07-14,SPY,EWJ,VGK,GLD,EEM
2023-06-13,EWJ,SPY,VGK,GLD,EEM
2023-05-12,VGK,EWJ,GLD,TLT,SPY
...,...,...,...,...,...
2007-11-05,EEM,DBC,GLD,TLT,VGK
2007-10-05,EEM,DBC,GLD,VGK,SPY
2007-09-06,EEM,VGK,GLD,SPY,DBC
2007-08-07,EEM,DBC,VGK,SPY,GLD


,1st,2nd,3rd,4th,5th
2023-09-13,14.8442,11.8201,9.3023,3.7070,2.5618
2023-08-14,10.6487,5.3659,4.6989,2.1343,-0.0822
2023-07-14,14.4493,13.8581,8.8700,3.8999,1.2277
2023-06-13,15.9809,11.9658,11.0324,8.0690,5.7809
2023-05-12,23.0301,18.2584,17.7498,11.8622,10.9395
...,...,...,...,...,...
2007-11-05,25.6880,21.2217,17.4867,5.1581,5.1296
2007-10-05,30.0572,10.5578,10.3262,9.0018,8.8786
2007-09-06,20.7142,9.1778,6.7928,6.1237,3.5543
2007-08-07,12.3499,6.9555,6.6743,2.8335,2.6084


'\n# 개수만큼 선정\nrow_count = df_etf_ror.shape[0]\nmonthly_index = df_etf_ror.index\n\ndf_high_etf = pd.DataFrame(columns=[i for i in range(row_count)], index=monthly_index)\n\nrow_count = len(df_high_etf)\n\nfor i in range(row_count):\n  df_high_etf.iloc[i] = df_etf_ror.columns\n\ndisplay(df_high_etf)\n'

In [ ]:
# 종목별 투자 비율
df_etf_monthly = df_etf[:-6*MBD:MBD]
display(df_etf_monthly)

monthly_row_count = df_high_etf.shape[0]
monthly_column_count = df_high_etf.shape[1]

# df_buy_ratio = pd.DataFrame(np.full((monthly_row_count, monthly_column_count), 1/monthly_column_count), columns=[i for i in range(monthly_column_count)], index=df_etf_ror.index)  # 동일 비중
# df_buy_ratio = etf_volatility_inverse_ratio  # 변동성 역비중
df_buy_ratio = etf_volatility_inverse_ratio_high  # top 5 변동성 역비중
display(df_buy_ratio)

,IEF,VNQ,GLD,DBC,EWJ,VGK,SPY,TLT,RWX,EEM
2023-09-13,93.5900,80.6600,177.0800,25.3800,62.2800,59.5700,446.5100,94.2100,24.7600,39.0200
2023-08-14,93.7401,82.7800,177.0600,24.3100,60.8200,61.5800,448.1100,94.8754,25.0900,39.3700
2023-07-14,96.0402,86.1300,181.4300,23.6400,62.4200,63.1700,449.2800,100.7095,26.0900,41.0100
2023-06-13,95.6905,82.3007,180.5400,22.6500,63.5100,61.1787,435.0440,100.3582,25.4894,40.3400
2023-05-12,98.4940,81.1141,186.8100,22.6700,59.7884,61.4637,410.0668,103.1184,26.0691,38.2503
...,...,...,...,...,...,...,...,...,...,...
2007-11-05,59.5782,34.8218,79.7500,28.3535,43.3226,42.5708,110.3513,57.9767,27.8437,37.3239
2007-10-05,58.1183,39.5217,73.4000,25.7464,45.6601,42.2657,114.6168,55.9975,28.9843,37.6427
2007-09-06,58.4533,35.3567,68.8600,24.0578,42.7927,39.7388,108.4249,56.2599,27.0128,32.4014
2007-08-07,57.0498,34.2090,66.4800,23.8258,44.3511,39.9583,108.1613,54.8856,26.9907,31.3949


,1st,2nd,3rd,4th,5th
2023-09-13,0.2256,0.1818,0.2346,0.1556,0.2024
2023-08-14,0.2493,0.1591,0.1673,0.2568,0.1675
2023-07-14,0.2591,0.1527,0.1744,0.2502,0.1636
2023-06-13,0.1747,0.2191,0.2113,0.2104,0.1845
2023-05-12,0.1795,0.2367,0.1924,0.1677,0.2236
...,...,...,...,...,...
2007-11-05,0.1069,0.1778,0.2147,0.3136,0.1869
2007-10-05,0.1081,0.2762,0.2462,0.1550,0.2145
2007-09-06,0.0973,0.1481,0.2694,0.1816,0.3036
2007-08-07,0.1116,0.2429,0.1795,0.1894,0.2766


In [ ]:
# 비율대로 돈 투자
df_invest_money = pd.DataFrame(columns=[i for i in range(monthly_column_count)], index=monthly_index)
df_invest_count = pd.DataFrame(columns=[i for i in range(monthly_column_count)], index=monthly_index)
df_cash = pd.DataFrame(columns=['cash'], index=monthly_index)

# 초기자본
invest_cash = 10_000

for i in reversed(range(row_count)):
  row_sum = 0

  for j in range(monthly_column_count):
    if i == row_count - 1:
      row_sum = invest_cash
      break
    else:
      sel_etf = df_high_etf.iloc[i+1][j]
      row_sum += df_invest_count.iloc[i+1][j] * df_etf_monthly.iloc[i][sel_etf]

  for j in range(monthly_column_count):
    df_cash.iloc[i][0] = row_sum
    sel_etf = df_high_etf.iloc[i][j]
    sel_etf_price = df_etf_monthly.iloc[i][sel_etf]
    sel_etf_buy_count = df_cash.iloc[i][0] * df_buy_ratio.iloc[i][j] / sel_etf_price
    df_invest_count.iloc[i][j] = sel_etf_buy_count

display(Markdown('### etf 구매 개수'))
display(df_invest_count)
display(Markdown('### 평가 금액'))
display(df_cash)

CAGR = str(round(((df_cash.iloc[0][0] / df_cash.iloc[-1][0]) ** (12/(row_count-1)) - 1)*100, 2))
display(Markdown(f'## CAGR: {CAGR} %'))

### etf 구매 개수

,0,1,2,3,4
2023-09-13,12.2923,71.0278,224.9771,97.0334,82.6830
2023-08-14,13.4706,63.3523,65.7923,35.1180,166.8861
2023-07-14,14.2716,60.5618,68.3148,34.1265,98.7551
2023-06-13,67.0617,12.2799,84.2054,28.4082,111.5021
2023-05-12,70.0451,94.9341,24.7056,39.0070,13.0797
...,...,...,...,...,...
2007-11-05,31.6624,69.3364,29.7614,59.7957,48.5306
2007-10-05,30.4814,113.8750,35.5994,38.9238,19.8584
2007-09-06,29.6479,36.8214,38.6433,16.5403,124.6386
2007-08-07,34.5656,99.1865,43.7065,17.0333,40.4720


### 평가 금액

,cash
2023-09-13,24333.8324
2023-08-14,24215.8628
2023-07-14,24749.1708
2023-06-13,24379.7843
2023-05-12,23982.3163
...,...
2007-11-05,11053.9137
2007-10-05,10613.5173
2007-09-06,9876.7630
2007-08-07,9727.7458


## CAGR: 5.65 %

In [ ]:
'''
# SPY, EFA 수익이 둘 다 BIL 보다 낮으면 채권에 투자
str_is_bond = 'False'
if ror_12mo['SPY'].iloc[0] < ror_12mo['BIL'].iloc[0] and ror_12mo['EFA'].iloc[0] < ror_12mo['BIL'].iloc[0]:
  str_is_bond = 'True'

# 채권 중 수익률 상위 3개에 각 33.3% 씩 투자! 그 중 6개월 수익률이 -이면 현금 보유
str_bond_top_3 = str_today + str_strategy + '### 6개월 수익률 안전자산 상위 3개'
ma_6mo_defensive_top_3 = ror_6mo[etf_defensive].transpose().nlargest(3, str(df_etf.index[0].date())).transpose()
display(Markdown(str_bond_top_3))
display(ma_6mo_defensive_top_3)

# 안전자산 중 0 이상인 것만 필터링하기
defensive_etf_list_text = ''

if str_is_bond == 'True':
  filtered_index = ma_6mo_defensive_top_3.columns[(ma_6mo_defensive_top_3 < 0).any()]
  df_etf_defensive_positive_index = ma_6mo_defensive_top_3.drop(filtered_index, axis=1).columns
  defensive_etf_list_text = ' '.join([str(asset) for asset in df_etf_defensive_positive_index])


## 결과 출력
display(Markdown(str_today[1:] + str_strategy))

str_etf = "## 12개월 수익률 기준, 미국 주식(SPY)과 글로벌 주식(EFA) 둘 다 초단기채(BIL) 보다 낮은가? " + str_is_bond
display(Markdown(str_etf))

# 아니라면 SPY, EFA 중 12개월 수익률이 더 높은 것 표시
str_higher_stock = ''

if str_is_bond == 'False':
  if ror_12mo['SPY'].iloc[0] > ror_12mo['EFA'].iloc[0]:
    str_higher_stock = 'SPY'
  else:
    str_higher_stock = 'EFA'

str_higher_stock_text = "## 투자할 공격자산: " + str_higher_stock
display(Markdown(str_higher_stock_text))

# 안전자산 중 6개월 수익률 0 이상 출력
str_etf_defensive_positive = '## 투자할 안전자산: ' + defensive_etf_list_text
display(Markdown(str_etf_defensive_positive))

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:] + str_strategy}\n')
  f.write(f'{str_etf}\n')
  f.write(f'{str_higher_stock_text}\n')
  f.write(f'{str_etf_defensive_positive}\n')
'''

'\n# SPY, EFA 수익이 둘 다 BIL 보다 낮으면 채권에 투자\nstr_is_bond = \'False\'\nif ror_12mo[\'SPY\'].iloc[0] < ror_12mo[\'BIL\'].iloc[0] and ror_12mo[\'EFA\'].iloc[0] < ror_12mo[\'BIL\'].iloc[0]:\n  str_is_bond = \'True\'\n\n# 채권 중 수익률 상위 3개에 각 33.3% 씩 투자! 그 중 6개월 수익률이 -이면 현금 보유\nstr_bond_top_3 = str_today + str_strategy + \'### 6개월 수익률 안전자산 상위 3개\'\nma_6mo_defensive_top_3 = ror_6mo[etf_defensive].transpose().nlargest(3, str(df_etf.index[0].date())).transpose()\ndisplay(Markdown(str_bond_top_3))\ndisplay(ma_6mo_defensive_top_3)\n\n# 안전자산 중 0 이상인 것만 필터링하기\ndefensive_etf_list_text = \'\'\n\nif str_is_bond == \'True\':\n  filtered_index = ma_6mo_defensive_top_3.columns[(ma_6mo_defensive_top_3 < 0).any()]\n  df_etf_defensive_positive_index = ma_6mo_defensive_top_3.drop(filtered_index, axis=1).columns\n  defensive_etf_list_text = \' \'.join([str(asset) for asset in df_etf_defensive_positive_index])\n\n\n## 결과 출력\ndisplay(Markdown(str_today[1:] + str_strategy))\n\nstr_etf = "## 12개월 수익률 기준, 미국 주식(SPY)과 글로